# Patients' Caregivers Accomodation in Bari
### A project for a more comfortable stay during hospitalization of patients


Background 
Hospitals in Bari are the reference for good medical care in Southern Italy. Every year thousands of patients, together with relatives and carigivers, come to Bari to undergo surgery or third level medical treatment. Nevertheless not every hospital neighborhoods are capable of providing a comfortable stay to patients' relatives and caregivers during hospitalization. 


Business
The Mayor of Bari requested an urban inquiry trying to unveil whether hospital neighborhoods requires new restaurant and/or accomodation to improve caregivers' stay.


Data
I will use a database containing all Hospitals in Bari (which may be accessed in my github) to provide a depiction of the most common services in their neighborhoods, paying particular attention on hotels and restaurants.




In [145]:
import requests
!pip install wget
import wget
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!pip install geopy
!pip install folium
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [157]:
url= 'https://github.com/drvincentvenus/My-Repository/blob/master/barihospital.xls?raw=true'

df1=pd.read_excel(url)

df1

,Denominazione ASL territoriale,Denominazione struttura,Indirizzo,CAP,Sigla provincia,Codice Comune,Natura giuridica,lat,long
0,ASL BA,CASA DI CURA SANTA MARIA,VIA A. DE FERRARIIS 18/D,70124,BA,72006,Privata,41.108405,16.861457
1,ASL BA,CASA DI CURA ANTHEA,VIA C. ROSALBA 35/37,70124,BA,72006,Privata,41.104302,16.857360
2,ASL BA,C.B.H. MATER DEI HOSPITAL,"VIA HAHNEMANN, 10",70125,BA,72006,Privata,41.097925,16.885576
3,ASL BA,OSPEDALE BARI 'SAN PAOLO',VIA CAPOSCARDICCHIO S.N.,70123,BA,72006,Pubblica,41.119219,16.777129
4,ASL BA,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),VIA OSPEDALE DI VENERE,70012,BA,72006,Pubblica,41.076839,16.863725
5,ASL BA,ISTITUTO TUMORI GIOVANNI PAOLO II,VIALE ORAZIO FLACCO N.65,70126,BA,72006,Pubblica,41.106603,16.858983
6,ASL BA,AO UNIV. CONS. POLICLINICO BARI,P.zza G. Cesare 11,70124,BA,72006,Pubblica,41.112282,16.862638


In [50]:
df1.rename(columns={'Denominazione ASL territoriale': 'ASL', 'CAP': 'Postal Code', 'Denominazione struttura': 'Hospital', 'Indirizzo': 'Address'}, inplace = True)
df1




,ASL,Hospital,Address,Postal Code,Sigla provincia,Codice Comune,Natura giuridica,lat,long
0,ASL BA,CASA DI CURA SANTA MARIA,VIA A. DE FERRARIIS 18/D,70124,BA,72006,Privata,41.108405,16.861457
1,ASL BA,CASA DI CURA ANTHEA,VIA C. ROSALBA 35/37,70124,BA,72006,Privata,41.104302,16.857360
2,ASL BA,C.B.H. MATER DEI HOSPITAL,"VIA HAHNEMANN, 10",70125,BA,72006,Privata,41.097925,16.885576
3,ASL BA,OSPEDALE BARI 'SAN PAOLO',VIA CAPOSCARDICCHIO S.N.,70123,BA,72006,Pubblica,41.119219,16.777129
4,ASL BA,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),VIA OSPEDALE DI VENERE,70012,BA,72006,Pubblica,41.076839,16.863725
5,ASL BA,ISTITUTO TUMORI GIOVANNI PAOLO II,VIALE ORAZIO FLACCO N.65,70126,BA,72006,Pubblica,41.106603,16.858983
6,ASL BA,AO UNIV. CONS. POLICLINICO BARI,P.zza G. Cesare 11,70124,BA,72006,Pubblica,41.112282,16.862638


In [51]:
CLIENT_ID = 'H2MOKQYZANUNAVFHHFBPYFQJZM5XC03HCFAYVA0GXCJSUDXA' #
CLIENT_SECRET = 'CQC3MTGLDAV0SR3IYAOGM1PFPWEAQ5XUTIXBRXV3KQCJED2X' 
VERSION = '20201102'

In [85]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hospital', 
                  'Hospital Latitude', 
                  'Hospital Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [68]:
hospital_venues = getNearbyVenues(names= df1['Hospital'],
                                   latitudes=df1['lat'],
                                   longitudes=df1['long']
                                   )

CASA DI CURA SANTA MARIA
CASA DI CURA ANTHEA
C.B.H. MATER DEI HOSPITAL
OSPEDALE BARI 'SAN PAOLO'
OSPEDALE BARI 'DI VENERE'  (TRIGGIANO)
ISTITUTO TUMORI GIOVANNI PAOLO II
AO UNIV. CONS. POLICLINICO BARI


In [86]:
hospital_venues

,Hospital,Hospital Latitude,Hospital Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CASA DI CURA SANTA MARIA,41.108405,16.861457,Factory,41.113323,16.864143,Gastropub
1,CASA DI CURA SANTA MARIA,41.108405,16.861457,Il Gelatiere,41.113689,16.863652,Ice Cream Shop
2,CASA DI CURA SANTA MARIA,41.108405,16.861457,Ekoinè Ri-pub,41.107765,16.863886,Pub
3,CASA DI CURA SANTA MARIA,41.108405,16.861457,Di Cosimo,41.105826,16.866903,Pizza Place
4,CASA DI CURA SANTA MARIA,41.108405,16.861457,Demetra,41.109601,16.863231,Cocktail Bar
5,CASA DI CURA SANTA MARIA,41.108405,16.861457,Sushi Tao,41.108108,16.867728,Japanese Restaurant
6,CASA DI CURA SANTA MARIA,41.108405,16.861457,Santa Scolastica,41.106742,16.866103,Market
7,CASA DI CURA SANTA MARIA,41.108405,16.861457,Hotel Mercure Villa Romanazzi Carducci,41.115737,16.859615,Hotel
8,CASA DI CURA SANTA MARIA,41.108405,16.861457,Windigo Saloon,41.103153,16.858356,Pub
9,CASA DI CURA SANTA MARIA,41.108405,16.861457,Storie Del Vecchio Sud,41.113052,16.870744,Pub


In [96]:
hospital_oneshot = pd.get_dummies(hospital_venues[['Venue Category']], prefix="", prefix_sep="")


In [99]:

hospital_oneshot.insert(loc=0, column='Hospital', value=hospital_venues['Hospital'])
hospital_oneshot.shape

(181, 46)

In [100]:
hospital_oneshot.head()

,Hospital,Accessories Store,Bakery,Bar,Beer Garden,Breakfast Spot,Brewery,Burger Joint,Business Service,Butcher,...,Pub,Restaurant,Sandwich Place,Shopping Mall,Soccer Field,Stadium,Steakhouse,Sushi Restaurant,Theater,Train Station
0,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
hospital_grouped = hospital_oneshot.groupby('Hospital').mean().reset_index()
hospital_grouped

,Hospital,Accessories Store,Bakery,Bar,Beer Garden,Breakfast Spot,Brewery,Burger Joint,Business Service,Butcher,...,Pub,Restaurant,Sandwich Place,Shopping Mall,Soccer Field,Stadium,Steakhouse,Sushi Restaurant,Theater,Train Station
0,AO UNIV. CONS. POLICLINICO BARI,0.00,0.025000,0.050000,0.02500,0.000000,0.025000,0.000000,0.000000,0.025000,...,0.125000,0.025000,0.000000,0.00,0.000000,0.00,0.025000,0.000000,0.00000,0.025000
1,C.B.H. MATER DEI HOSPITAL,0.00,0.000000,0.074074,0.00000,0.037037,0.037037,0.037037,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.037037,0.00,0.037037,0.037037,0.00000,0.000000
2,CASA DI CURA ANTHEA,0.00,0.027778,0.055556,0.00000,0.000000,0.000000,0.000000,0.000000,0.027778,...,0.111111,0.000000,0.027778,0.00,0.000000,0.00,0.000000,0.027778,0.00000,0.027778
3,CASA DI CURA SANTA MARIA,0.00,0.048780,0.048780,0.02439,0.000000,0.024390,0.000000,0.000000,0.024390,...,0.146341,0.024390,0.000000,0.00,0.000000,0.00,0.024390,0.000000,0.02439,0.024390
4,ISTITUTO TUMORI GIOVANNI PAOLO II,0.00,0.033333,0.066667,0.00000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.133333,0.033333,0.033333,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.033333
5,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),0.25,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.25,0.000000,0.25,0.000000,0.000000,0.00000,0.000000
6,OSPEDALE BARI 'SAN PAOLO',0.00,0.333333,0.000000,0.00000,0.000000,0.000000,0.000000,0.333333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00000,0.000000


In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hospital']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hospital_venues_sorted = pd.DataFrame(columns=columns)
hospital_venues_sorted['Hospital'] = hospital_grouped['Hospital']

for ind in np.arange(hospital_grouped.shape[0]):
    hospital_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hospital_grouped.iloc[ind, :], num_top_venues)

hospital_venues_sorted

,Hospital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AO UNIV. CONS. POLICLINICO BARI,Pub,Hotel,Pizza Place,Café,Fast Food Restaurant,Bar,Japanese Restaurant,Butcher,Electronics Store,Hostel
1,C.B.H. MATER DEI HOSPITAL,Pizza Place,Café,Bar,Gym,Park,Burger Joint,Movie Theater,Multiplex,Noodle House,Electronics Store
2,CASA DI CURA ANTHEA,Pizza Place,Pub,Japanese Restaurant,Café,Bar,Train Station,Ice Cream Shop,Market,Grocery Store,Gastropub
3,CASA DI CURA SANTA MARIA,Pizza Place,Pub,Café,Bar,Outdoors & Recreation,Bakery,Japanese Restaurant,Electronics Store,Hotel,Ice Cream Shop
4,ISTITUTO TUMORI GIOVANNI PAOLO II,Pizza Place,Pub,Japanese Restaurant,Café,Bar,Train Station,Hotel,Ice Cream Shop,Market,Gastropub
5,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),Accessories Store,Stadium,Shopping Mall,Cocktail Bar,Chinese Restaurant,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub
6,OSPEDALE BARI 'SAN PAOLO',Kitchen Supply Store,Bakery,Business Service,Cocktail Bar,Hotel,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub


In [123]:
# set number of clusters
kclusters = 4

hospital_grouped_clustering = hospital_grouped.drop('Hospital', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hospital_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 3, 0, 0, 0, 2, 1])

In [133]:
# add clustering labels
hospital_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
hospital_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hospital_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hospital_merged = hospital_merged.join(hospital_venues_sorted.set_index('Hospital'), on='Hospital')

hospital_merged.head()

,ASL,Hospital,Address,Postal Code,Sigla provincia,Codice Comune,Natura giuridica,lat,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ASL BA,CASA DI CURA SANTA MARIA,VIA A. DE FERRARIIS 18/D,70124,BA,72006,Privata,41.108405,16.861457,0,Pizza Place,Pub,Café,Bar,Outdoors & Recreation,Bakery,Japanese Restaurant,Electronics Store,Hotel,Ice Cream Shop
1,ASL BA,CASA DI CURA ANTHEA,VIA C. ROSALBA 35/37,70124,BA,72006,Privata,41.104302,16.857360,0,Pizza Place,Pub,Japanese Restaurant,Café,Bar,Train Station,Ice Cream Shop,Market,Grocery Store,Gastropub
2,ASL BA,C.B.H. MATER DEI HOSPITAL,"VIA HAHNEMANN, 10",70125,BA,72006,Privata,41.097925,16.885576,3,Pizza Place,Café,Bar,Gym,Park,Burger Joint,Movie Theater,Multiplex,Noodle House,Electronics Store
3,ASL BA,OSPEDALE BARI 'SAN PAOLO',VIA CAPOSCARDICCHIO S.N.,70123,BA,72006,Pubblica,41.119219,16.777129,1,Kitchen Supply Store,Bakery,Business Service,Cocktail Bar,Hotel,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub
4,ASL BA,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),VIA OSPEDALE DI VENERE,70012,BA,72006,Pubblica,41.076839,16.863725,2,Accessories Store,Stadium,Shopping Mall,Cocktail Bar,Chinese Restaurant,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub


In [135]:
hospital_venues_sorted

,Cluster Labels,Hospital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,AO UNIV. CONS. POLICLINICO BARI,Pub,Hotel,Pizza Place,Café,Fast Food Restaurant,Bar,Japanese Restaurant,Butcher,Electronics Store,Hostel
1,3,C.B.H. MATER DEI HOSPITAL,Pizza Place,Café,Bar,Gym,Park,Burger Joint,Movie Theater,Multiplex,Noodle House,Electronics Store
2,0,CASA DI CURA ANTHEA,Pizza Place,Pub,Japanese Restaurant,Café,Bar,Train Station,Ice Cream Shop,Market,Grocery Store,Gastropub
3,0,CASA DI CURA SANTA MARIA,Pizza Place,Pub,Café,Bar,Outdoors & Recreation,Bakery,Japanese Restaurant,Electronics Store,Hotel,Ice Cream Shop
4,0,ISTITUTO TUMORI GIOVANNI PAOLO II,Pizza Place,Pub,Japanese Restaurant,Café,Bar,Train Station,Hotel,Ice Cream Shop,Market,Gastropub
5,2,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),Accessories Store,Stadium,Shopping Mall,Cocktail Bar,Chinese Restaurant,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub
6,1,OSPEDALE BARI 'SAN PAOLO',Kitchen Supply Store,Bakery,Business Service,Cocktail Bar,Hotel,Hostel,Gym / Fitness Center,Gym,Grocery Store,Gastropub


In [136]:
address = 'Bari, IT'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of BARI are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of BARI are 41.1257843, 16.8620293.


In [138]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hospital_merged['lat'], hospital_merged['long'], hospital_merged['Hospital'], hospital_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Conclusion:

Hospital neighborhoods in Cluster 0-1 offer plenty of hotels and restaurants. 

Hospital neighborhood in Cluster 2 needs more reataurants. 

Hospital neighborhood in Cluster 3 needs more hotels.

    
